In [51]:
import torch 
from torch import nn
import numpy as np


# text="sriram kumar is a good boy. this is alpha beta gamma"

chunk_size = 10000
# Read the text data from the file
file_path = './books/harry_potter.txt'  # Update the path to your book file
with open(file_path, 'r') as file:
    book_text = file.read(chunk_size)

# Convert the text to lowercase
book_text_lower = book_text.lower()

# You can optionally perform additional cleaning or preprocessing here if needed,
# such as removing special characters, punctuation, or performing tokenization.

# Example: Removing punctuation using regex
import re
text =  re.sub(r'[^a-zA-Z\s]', '', book_text_lower) #re.sub(r'[^\w\s]', '', book_text_lower)

# Now you have the cleaned text in lowercase. You can use book_text_cleaned for further processing.


# Create Character Mappings 
chars = sorted(list(set(text)))
n_chars = len(chars)
char_to_index = {ch:i for i,ch in enumerate(chars)}
index_to_char = {i:ch for i,ch in enumerate(chars)}

print(char_to_index)
print(n_chars)

def one_hot_encode(char):
    vector = np.zeros(n_chars)

    vector[char_to_index[char]] = 1
    return vector

# Prepare input sequence and corresponding target characters 
seq_length = 10
X_data, y_data = [] , []

for i in range(0,len(text)-seq_length,1):
    seq_in =  text[i:i+seq_length]
    seq_out = text[i+seq_length]
    X_data.append([one_hot_encode(char) for char in seq_in ])
    y_data.append([one_hot_encode(seq_out)])

# Convert data to pytorch tensors 
X = torch.tensor(X_data, dtype=torch.float32)
# y = torch.tensor(np.array(y_data).squeeze())

y = torch.tensor(np.array(y_data).squeeze(), dtype=torch.long)  # Change the data type to torch.long

print( f""" Total training data count {X.shape[0]}  uniquechars aka input vector {y.shape[1]} ,
       X {X.shape} , y {y.shape} 
       for Each x of seq of {X.shape[1]} characters , one y target character 
       Each  character is of {y.shape[1]} dimensional vector """)

{'\n': 0, ' ': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27}
28
 Total training data count 9595  uniquechars aka input vector 28 ,
       X torch.Size([9595, 10, 28]) , y torch.Size([9595, 28]) 
       for Each x of seq of 10 characters , one y target character 
       Each  character is of 28 dimensional vector 


# Model 

In [52]:


import torch
import torch.nn as nn
torch.manual_seed(42) # For reproducibility
hidden_size = 32
# model = nn.Sequential(
#     nn.RNN(n_chars, hidden_size),  # RNN layer
#     nn.Linear(hidden_size, n_chars)  # Output layer mapping RNN hidden representation to character space
# )



class CharGenerationRNN(nn.Module):
    def __init__(self, n_chars, hidden_size):
        super().__init__()
        self.rnn = nn.RNN(n_chars, hidden_size)
        self.lin = nn.Linear(hidden_size, n_chars)

    def forward(self, x):
        # Forward pass through the RNN layer.
        output, hidden = self.rnn(x)
        
        # Extract the output at the last time step.
        last_output = output
        
        # Pass the last RNN output through the linear layer to get the final output.
        return self.lin(last_output)



model = CharGenerationRNN(n_chars,hidden_size)

def sample_size_shape_test():
    input = X[0][0:4]
    
    # print(f"input {input} shape {input.shape} type {type(input)} ")
    hidden_unit_ouput = model(input)
    # print(output)
    print(hidden_unit_ouput.shape)
    print([ index_to_char[int(torch.argmax(one_hot_vector_char))] for one_hot_vector_char in input])
    print(f" hidden shape {hidden_unit_ouput} ")
    print(f" hidden shape {index_to_char[int(torch.argmax(hidden_unit_ouput[-1]))]} ")
    # print("^ Last Hidden state is the ouput ")


sample_size_shape_test()


torch.Size([4, 28])
['\n', '\n', 'w', 'h']
 hidden shape tensor([[ 0.0246, -0.0048, -0.0150, -0.1198, -0.2089,  0.0909,  0.1411, -0.2366,
         -0.0254,  0.0137,  0.0608, -0.0524,  0.0273, -0.0580,  0.1160,  0.0740,
          0.1266,  0.0246,  0.1072,  0.1110, -0.0395, -0.0713, -0.0220,  0.1484,
          0.0124, -0.0821, -0.2595,  0.1276],
        [ 0.0755,  0.0163, -0.0047, -0.2126, -0.2976,  0.0668,  0.1895, -0.2509,
         -0.0398,  0.1024,  0.0931, -0.1112,  0.0048, -0.0547,  0.0726,  0.0544,
          0.1044,  0.0051,  0.0590,  0.1670, -0.1144, -0.0211, -0.0870,  0.0977,
          0.0566, -0.1103, -0.2327,  0.1807],
        [ 0.1926,  0.0807, -0.0600, -0.1992, -0.4037, -0.0175,  0.1228, -0.2418,
         -0.0737,  0.0659,  0.1062, -0.1004, -0.0020,  0.0242,  0.1004,  0.0890,
          0.0151, -0.0237,  0.0435,  0.1044, -0.0525, -0.0218, -0.1134, -0.0197,
         -0.0248, -0.1781, -0.2917,  0.1902],
        [ 0.0236,  0.0232,  0.0340, -0.1386, -0.2281,  0.0480,  0.1940, -0.3

In [53]:

# print(model.state_dict())

# Define loss and optimizer
criteria = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)



X.shape , y.shape

(torch.Size([9595, 10, 28]), torch.Size([9595, 28]))

# Training

In [72]:
looses = []
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(X)
    # print(X.shape)
    # print("------")
    y_pred = output[:,-1,:]
    # print(y_pred.shape)
    # print("------")
    # print(y.shape)
    # print("------")
    loss = criteria(y_pred.float(),y.float())
    loss.backward()
    optimizer.step()
    looses.append(loss.item())
    if epoch%1 == 0:
        print(f"epoch {epoch} loss : {loss.item()}")

epoch 0 loss : 1.808928370475769
epoch 1 loss : 1.8046586513519287
epoch 2 loss : 1.8007177114486694
epoch 3 loss : 1.7978651523590088
epoch 4 loss : 1.797671914100647
epoch 5 loss : 1.799382209777832
epoch 6 loss : 1.795212745666504
epoch 7 loss : 1.79088294506073
epoch 8 loss : 1.7889983654022217
epoch 9 loss : 1.7864056825637817
epoch 10 loss : 1.7854559421539307
epoch 11 loss : 1.7872754335403442
epoch 12 loss : 1.7874181270599365
epoch 13 loss : 1.783036231994629
epoch 14 loss : 1.7797266244888306
epoch 15 loss : 1.7767844200134277
epoch 16 loss : 1.7740968465805054
epoch 17 loss : 1.774444341659546
epoch 18 loss : 1.7760969400405884
epoch 19 loss : 1.7732905149459839
epoch 20 loss : 1.7695956230163574
epoch 21 loss : 1.7678040266036987
epoch 22 loss : 1.7644520998001099
epoch 23 loss : 1.7616122961044312
epoch 24 loss : 1.762022852897644
epoch 25 loss : 1.7647000551223755
epoch 26 loss : 1.765826940536499
epoch 27 loss : 1.76207435131073
epoch 28 loss : 1.7577074766159058
epoch 2

In [63]:

sample_size_shape_test()

torch.Size([4, 28])
['\n', '\n', 'w', 'h']
 hidden shape tensor([[-1.2675, -1.7056,  1.7738,  2.1560,  1.9441,  1.6303, -2.2309,  1.9248,
          1.8374,  2.5438,  0.7526,  0.4009, -0.8015,  0.1052,  1.3983, -0.5474,
          1.2071,  1.1043, -1.0536, -1.2484,  1.7843,  2.4987, -0.9531, -0.6692,
          2.2068, -2.1728, -3.5215, -1.3798],
        [-0.9145, -1.9017,  2.9679,  0.1800,  0.6945, -2.3314, -1.9408,  3.4801,
         -1.7673,  3.6643,  0.4927,  0.4204, -2.2974,  0.4703,  0.3772,  1.2529,
          0.9932,  0.6764,  0.3370,  1.0761,  1.2819,  2.0112,  1.1288, -0.5162,
          1.3360, -0.4055, -4.6875, -0.4544],
        [ 0.2659,  1.3589,  2.2608, -0.8085,  0.2920, -3.9356,  0.3886,  1.2563,
         -5.0885,  2.2399,  0.7852, -1.4836, -2.6735,  4.2599,  0.0565, -2.9677,
          1.1422, -0.2652, -1.8120,  0.3384, -0.0619,  1.7933, -0.2971, -0.2966,
          0.9539, -1.0676, -3.7285, -1.2522],
        [-0.5125,  2.2139,  5.1599, -0.2908,  0.7290, -4.0583,  6.4634, -1.6

In [73]:
import random
from time import sleep
from IPython import display

# Use Trained model to geenrate Data 
def String2Vec(string):
    X_data=[]
    X_data.append(([one_hot_encode(c) for idx,c in enumerate(string) ]))
    return X_data

def generativeAI(input_string,n_output_chars):
    genText = input_string
    # Convert data to pytorch tensors 
    X = torch.tensor(String2Vec(input_string), dtype=torch.float32).squeeze()

    for i in range(n_output_chars):
        hidden_unit_ouput = model(X)

        # print(hidden_unit_ouput)
        # Get the indices of the top 3 maximum values
        topk_values, topk_indices = torch.topk(hidden_unit_ouput[:,-1].squeeze(), k=3)
        print(topk_indices)
        # Choose randomly from the top 3 indices
        random_index = random.choice(topk_indices)
        nextCharIndex = int(random_index)

        # nextCharIndex = int(torch.argmax(hidden_unit_ouput))
        nextChar = index_to_char[nextCharIndex]
        nextCharOneHot = torch.tensor(one_hot_encode(nextChar), dtype=torch.float32).unsqueeze(dim=0)
        # torch.hstack(X,one_hot_encode(index_to_char(nextChar)))
        genText = genText+nextChar
        print(genText)
        sleep(0.5)
        if i != n_output_chars-1:
            display.clear_output(wait=True)
        X = torch.vstack((X,nextCharOneHot))
   
    
generativeAI('har',30)

tensor([14,  5, 21])
har a bdedbbjjeddmdddemdmmdtmdttm


# FROM CHATGPT

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim

# Load the dataset from a text file
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()
    return data

# Preprocess the data
def preprocess_data(data):
    unique_chars = list(set(data))
    char_to_index = {char: i for i, char in enumerate(unique_chars)}
    index_to_char = {i: char for i, char in enumerate(unique_chars)}
    return char_to_index, index_to_char

# Generate training pairs
def generate_training_pairs(data, sequence_length=100):
    input_sequences = []
    target_chars = []
    for i in range(0, len(data) - sequence_length, 1):
        input_seq = data[i:i + sequence_length]
        target_char = data[i + sequence_length]
        input_sequences.append(input_seq)
        target_chars.append(target_char)
    return input_sequences, target_chars

# Create the CharGenerationRNN model
class CharGenerationRNN(nn.Module):
    def __init__(self, n_chars, hidden_size):
        super().__init__()
        self.rnn = nn.RNN(n_chars, hidden_size, batch_first=True)
        self.lin = nn.Linear(hidden_size, n_chars)

    def forward(self, x):
        output, _ = self.rnn(x)
        output = output[:, -1, :]  # Take the output from the last time step
        output = self.lin(output)
        return output

# Training parameters
file_path = './books/harry_potter.txt'  # Replace with the path to your text file
sequence_length = 100
hidden_size = 128
learning_rate = 0.001
epochs = 10

# Load and preprocess the data
data = load_data(file_path)
char_to_index, index_to_char = preprocess_data(data)

# Generate training pairs
input_sequences, target_chars = generate_training_pairs(data, sequence_length)

# Convert characters to indices
input_indices = [[char_to_index[char] for char in seq] for seq in input_sequences]
target_indices = [char_to_index[char] for char in target_chars]

# Convert to PyTorch tensors
input_tensors = torch.tensor(input_indices, dtype=torch.long)
target_tensors = torch.tensor(target_indices, dtype=torch.long)

# Create the model
model = CharGenerationRNN(len(char_to_index), hidden_size)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    for i in range(len(input_tensors)):
        input_seq = input_tensors[i].view(-1, 1, len(char_to_index)).permute(1, 0, 2)
        target_char = target_tensors[i]

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(input_seq)

        # Calculate the loss
        loss = criterion(output, target_char.unsqueeze(0))

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

# Generating text
def generate_text(starting_sequence, model, char_to_index, index_to_char, max_length=100):
    model.eval()
    with torch.no_grad():
        current_sequence = starting_sequence
        for _ in range(max_length):
            # Convert the current sequence to indices
            sequence_indices = [char_to_index[char] for char in current_sequence]
            input_tensor = torch.tensor(sequence_indices, dtype=torch.long).view(-1, 1, len(char_to_index)).permute(1, 0, 2)

            # Forward pass
            output = model(input_tensor)

            # Get the predicted next character
            _, predicted_index = torch.max(output, 1)
            predicted_char = index_to_char[predicted_index.item()]

            # Append the predicted character to the sequence
            current_sequence += predicted_char

        return current_sequence

# Example usage
starting_sequence = "To be or not to b"
generated_text = generate_text(starting_sequence, model, char_to_index, index_to_char)
print(generated_text)


RuntimeError: shape '[-1, 1, 85]' is invalid for input of size 100